In [ ]:
pip install geopandas plotly dash pandas openpyxl shapely

In [ ]:
pip install dash-bootstrap-components

In [2]:
import geopandas as gpd
world = gpd.read_file("Shape_file_10m/ne_10m_admin_0_countries.shp")
world.columns

Index(['featurecla', 'scalerank', 'LABELRANK', 'SOVEREIGNT', 'SOV_A3',
       'ADM0_DIF', 'LEVEL', 'TYPE', 'TLC', 'ADMIN',
       ...
       'FCLASS_TR', 'FCLASS_ID', 'FCLASS_PL', 'FCLASS_GR', 'FCLASS_IT',
       'FCLASS_NL', 'FCLASS_SE', 'FCLASS_BD', 'FCLASS_UA', 'geometry'],
      dtype='object', length=169)

In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import io, json

# === 1. Load Geo Data ===
world = gpd.read_file("Shape_file_10m/ne_10m_admin_0_countries.shp")[["SOV_A3", "ADMIN", "geometry"]]
world = world.rename(columns={"SOV_A3": "uCode", "ADMIN": "Country"})
world_json = json.loads(world.to_json())

# === 2. Load Excel (multi-sheets) ===
xls = pd.read_excel("GHI_CoinR.xlsx", sheet_name=["iData_Short", "iData_Mid", "iData_Long"])

# clean each df: keep consistent colnames
for k in xls:
    xls[k] = xls[k].rename(columns={"uName": "Country"})

# === 3. Load Meta Data ===
meta = pd.read_excel("GHI_CoinR.xlsx", sheet_name='iMeta')
meta_dict = dict(zip(meta["iCode"], meta["IndName"]))

# === 4. Dash App ===
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Interactive Atlas"),

    html.Div([
        html.Label("Select Scenario"),
        dcc.Dropdown(
            id="scenario",
            options=[{"label": k.replace("iData_", ""), "value": k} for k in xls.keys()],
            value="iData_Short"
        ),

        html.Label("Select Indicator"),
        dcc.Dropdown(id="indicator"),
    ], style={"width": "40%", "display": "inline-block", "verticalAlign": "top"}),

    dcc.Graph(id="map", style={"height": "600px"}),

    html.Div(id="note", style={"marginTop": "20px", "fontStyle": "italic"}),

    html.Div(id="selected-country", style={"marginTop": "20px", "fontWeight": "bold"}),

    html.Button("Download Country Profile", id="download-btn"),
    dcc.Download(id="download")
])

# === 5. Callbacks ===

# Update indicator list based on scenario
@app.callback(
    Output("indicator", "options"),
    Input("scenario", "value")
)
def update_indicator_options(scenario):
    df = xls[scenario]
    indicators = [col for col in df.columns if col not in ["uCode", "Country"]]
    return [{"label": meta_dict.get(ind, ind), "value": ind} for ind in indicators]

# Update map
@app.callback(
    Output("map", "figure"),
    [Input("scenario", "value"),
     Input("indicator", "value")]
)
def update_map(scenario, indicator):
    df = xls[scenario]
    merged = world.merge(df, on="uCode", how="left")

    fig = px.choropleth(
        merged,
        geojson=world_json,
        locations="uCode",
        featureidkey="properties.uCode",
        color=indicator,
        hover_name="Country",
        projection="mercator",
        title=meta_dict.get(indicator, indicator) if indicator else ""
    )
    fig.update_geos(fitbounds="locations", visible=False)

    return fig

# Store selected country from click
@app.callback(
    Output("selected-country", "children"),
    Input("map", "clickData")
)
def select_country(clickData):
    if clickData:
        country = clickData["points"][0]["hovertext"]
        return f"Selected Country: {country}"
    return "Click on a country to see its profile"

# Download country profile (all scenarios, all indicators)
@app.callback(
    Output("download", "data"),
    Input("download-btn", "n_clicks"),
    Input("map", "clickData"),
    prevent_initial_call=True
)
def download_country_profile(n_clicks, clickData):
    if not clickData:
        return None
    country_name = clickData["points"][0]["hovertext"]

    profiles = []
    for scenario, df in xls.items():
        row = df[df["Country"] == country_name]
        if not row.empty:
            row = row.copy()
            row["Scenario"] = scenario.replace("iData_", "")
            profiles.append(row)

    if not profiles:
        return None

    out = pd.concat(profiles, ignore_index=True)

    buf = io.BytesIO()
    out.to_excel(buf, index=False, engine="openpyxl")
    buf.seek(0)

    return dict(content=buf.getvalue(), filename=f"profile_{country_name}.xlsx")

# === Run App ===
if __name__ == "__main__":
    app.run(debug=True)
